# **Step1_AI면접관 Agent v1.0**

## **1. 환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project_genai)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### (2) 라이브러리

In [11]:
!pip install -r /content/drive/MyDrive/project_genai/requirements.txt -q

### (3) OpenAI API Key 확인
* api_key.txt 파일에 다음의 키를 등록하세요.
    * OPENAI_API_KEY
    * NGROK_AUTHTOKEN

In [12]:
import os

def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path = '/content/drive/MyDrive/project_genai/'
# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

In [13]:
print(os.environ['OPENAI_API_KEY'][:30])

sk-proj-hzRpWhet6v1j6AyC_oUyrM


## **2. App.py**

* 아래 코드에, Step1 혹은 고도화 된 Step2 파일 코드를 붙인다.
    * 라이브러리
    * 함수들과 그래프
* Gradio 코드는 그대로 사용하거나 일부 수정 가능

In [14]:
%%writefile app.py

####### 여러분의 함수와 클래스를 모드 여기에 붙여 넣읍시다. #######
## 1. 라이브러리 로딩 ---------------------------------------------
import pandas as pd
import numpy as np
import os
import openai
import random
import ast
import fitz
from docx import Document as DocxDocument

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from typing import Annotated, Literal, Sequence, TypedDict, List, Dict
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.output_parsers import CommaSeparatedListOutputParser, PydanticOutputParser
from langgraph.graph import StateGraph, START, END

from pydantic import BaseModel

## ---------------- 1단계 : 사전준비 ----------------------

# 1) 파일 입력 --------------------
def extract_text_from_file(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        doc = fitz.open(file_path)
        text = "\n".join(page.get_text() for page in doc)
        doc.close()
        return text
    elif ext == ".docx":
        doc = DocxDocument(file_path)
        return "\n".join(p.text for p in doc.paragraphs if p.text.strip())
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 허용됩니다.")

# 2) State 선언 --------------------
class StrategySection(BaseModel):
    질문방향: str
    예시질문: list[str]

class InterviewStrategy(BaseModel):
    경력_및_경험: StrategySection
    동기_및_커뮤니케이션: StrategySection
    논리적_사고: StrategySection

class Evaluation(BaseModel):
    질문과의_관련성: str
    답변의_구체성: str

class InterviewState(TypedDict):
    # 고정 정보
    resume_text: str
    resume_summary: str
    resume_keywords: List[str]
    question_strategy: InterviewStrategy

    # 인터뷰 로그
    current_question: str
    current_answer: str
    current_strategy: str
    conversation: List[Dict[str, str]]
    evaluation : List[Evaluation]
    strategies: List[str]
    next_step : str
    #######################################################
    evaluation_ok: bool
    re_evaluated: bool
    #######################################################

    ## 1. 수정 부분
    feedback_report: str

# 3) resume 분석 --------------------
def analyze_resume(state: InterviewState) -> InterviewState:
    # 여기에 코드를 완성합니다.

    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

    summary_prompt = PromptTemplate(
        input_variables=["resume"],
        template="""
        {resume}를 읽고,
        개인화된 질문을 뽑기 위한 이력서 핵심 내용만 보고서 형태로 추출해줘.
        앞뒤 부가내용 없이"""
    )
    summary_chain = summary_prompt | llm | StrOutputParser()

    keyword_prompt = PromptTemplate(
        input_variables=["resume"],
        template="""
        {resume}를 읽고,
        주요 키워드만 쉼표로 구분해서 10개 내외로 뽑아줘.
        부가설명 없이 키워드만."""
    )
    keyword_chain = keyword_prompt | llm | StrOutputParser()


    resume_summary = summary_chain.invoke({"resume": state['resume_text']})

    keywords_str = keyword_chain.invoke({"resume": state['resume_text']})
    resume_keywords = [kw.strip() for kw in keywords_str.split(",")]

    # return 코드는 제공합니다.
    return {
        **state,
        "resume_summary": resume_summary,
        "resume_keywords": resume_keywords,
    }

# 4) 질문 전략 수립 --------------------
def generate_question_strategy(state: InterviewState) -> InterviewState:
    # 여기에 코드를 완성합니다.
    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

    prompt = PromptTemplate(
            input_variables=["resume_summary", "resume_keywords"],
            template="""
            [이력서 요약]
            {resume_summary}

            [주요 키워드]
            {resume_keywords}

            -- 면접 전략 생성
            - 다음 3개 영역별로 JSON 형식 생성:
            1. 경력 및 경험: 역할/성과 중심 질문
            2. 동기 및 커뮤니케이션: 가치관/소통능력 평가
            3. 논리적 사고: 문제해결 과정 평가

            -- 출력 형식
            {{
              "경력_및_경험": {{
                "질문방향": "...",
                "예시질문": ["...", "..."]
              }},
              "동기_및_커뮤니케이션": {{
                "질문방향": "...",
                "예시질문": ["...", "..."]
              }},
              "논리적_사고": {{
                "질문방향": "...",
                "예시질문": ["...", "..."]
              }}
            }}
            """
        )

    # 체인 구성
    parser = PydanticOutputParser(pydantic_object=InterviewStrategy)
    chain = prompt | llm | parser

    strategy = chain.invoke({
        "resume_summary": state['resume_summary'],
        "resume_keywords": state['resume_keywords']
    })


    # return 코드는 제공합니다.
    return {
        **state,
        "question_strategy": strategy
    }

def makeState(resume_text: str) -> InterviewState:
  return {
      "resume_text": resume_text,
      "resume_summary": '',
      "resume_keywords": [],
      "question_strategy": {},

      "current_question": '',
      "current_answer": '',
      "current_strategy": '',
      "conversation": [],
      "evaluation": [],
      "strategies": [],
      "next_step" : '',
      #######################################################
      "evaluation_ok": False,
       "re_evaluated": False,
      #######################################################

      # 1. 수정 부분
      "feedback_report": '',
  }

# 5) 1단계 하나로 묶기 --------------------

def preProcessing_Interview(file_path: str) -> InterviewState:
    # 여기에 코드를 완성합니다.
    resume_text = extract_text_from_file(file_path)
    initial_state = makeState(resume_text)

    analyzed_state = analyze_resume(initial_state)
    state = generate_question_strategy(analyzed_state)

    strategy = state['question_strategy']

    selected_question = strategy.경력_및_경험.예시질문[0]


    # return 코드는 제공합니다.
    return {
            **state,
            "current_question": selected_question,
            "current_strategy": "경력 및 경험"
            }


## ---------------- 2단계 : 면접 Agent ----------------------

# 1) 답변 입력 --------------------
def update_current_answer(state: InterviewState, user_answer: str) -> InterviewState:
    return {
        **state,
        "current_answer": user_answer.strip()
    }

# 2) 답변 평가 --------------------
def evaluate_answer(state: InterviewState) -> InterviewState:
    # 여기에 코드를 완성합니다.
    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

    prompt = PromptTemplate(
            input_variables=["current_question", "current_answer"],
            template="""
            [질문]
            {current_question}

            [답변]
            {current_answer}

            -- 다음 항목에 대한 답변 평가
            - '상', '중', '하' 로 평가
            - 1. 질문과의 관련성
              - '상': 질문의 핵심 의도 정확히 부함하며, 전반적인 내용을 명확히 다룸
              - '중': 질문과 관련은 있지만 핵심 포인트가 부분적으로 누락됨
              - '하': 질문과 관련이 약하거나 엉뚱한 내용 중심
            - 2. 답변의 구체성
              - '상': 구체적인 사례, 데이터, 단계별 설명 등이 포함되어 명확히 근거를 제시함
              - '중': 일부 구체적 내용이 있으나 추가 설명이나 예시가 부족함
              - '하': 추상적이거나 모호한 표현만 사용되어 핵심 내용이 불분명함

            -- 출력 형식
            {{
              "질문과의_관련성": "...",
              "답변의_구체성": "..."
            }}
            """
        )

    # 체인 구성
    parser = PydanticOutputParser(pydantic_object=Evaluation)
    chain = prompt | llm | parser

    assessment = chain.invoke({
        "current_question": state['current_question'],
        "current_answer": state['current_answer']
    })

    state['conversation'].append({
        "question": state['current_question'],
        "answer": state['current_answer']
    })


    state['evaluation'].append(assessment)
    state['strategies'].append(state['current_strategy'])

    #######################################################
    state['evaluation_ok'] = (
        assessment.질문과의_관련성 == '상'
        and assessment.답변의_구체성    == '상'
    )
    if 're_evaluated' not in state:
        state['re_evaluated'] = False
    #######################################################
    # return 코드는 제공합니다.

    return {
        **state
    }

# 3) 인터뷰 진행 검토 --------------------
def decide_next_step(state: InterviewState) -> InterviewState:
    # 여기에 코드를 완성합니다.

    keys = [key.replace("_", " ") for key in InterviewStrategy.__fields__.keys()]
    latest_eval = state['evaluation'][-1]

    if (all(item in state['strategies'] for item in keys)):
      # log
      print("="*60)
      print("[decide_next_step] 🚩 모든 전략 영역을 커버했음")
      print("="*60)

      next_step = 'end'

    elif (len(state['conversation']) >= 5):
      next_step = 'end'

    elif ((latest_eval.질문과의_관련성 == '하') or (latest_eval.답변의_구체성 == '하')):
      # log
      print("="*60)
      print("[decide_next_step] 🚩 답변 평가가 '하'로 추가 질문을 생성합니다.")
      print("="*60)

      next_step = 'additional_question'

    else:
      state['current_strategy'] = next(
          (k for k in keys if k not in state['strategies']),
          None)

      # log
      print("="*60)
      print(f"[decide_next_step] 🚩 질문 전략을 변경합니다. {state['current_strategy']}")
      print("="*60)

      next_step = 'additional_question'


    # return 코드는 제공합니다.
    return {
        **state,
        "next_step": next_step
    }

# 4) 질문 생성 --------------------
question_db = [
    # 1. 경력 및 경험
    "KT AI 연구소 인턴십에서 딥러닝 후처리 파이프라인 설계 시 가장 중점을 둔 기술적 요소는 무엇이었나요?",
    "OCR 성능을 12% 개선하는 과정에서 어떤 문제가 가장 도전적이었으며, 이를 어떻게 해결했나요?",
    "AI 면접관 시스템을 개발하면서 GPT와 FAISS를 연동한 방식에 대해 설명해 주세요.",
    "빅데이터 학생연합 기술부장 시절, 팀 리더로서 기술적 의사결정을 내린 경험이 있다면 설명해 주세요.",
    "부동산 가격 예측 프로젝트에서 공공데이터 활용 과정에서 겪은 어려움과 해결방안을 공유해 주세요.",
    "Streamlit을 활용한 프론트엔드 구성에서 사용자 경험(UX)을 어떻게 고려했는지 설명해 주세요.",
    "딥러닝 기반 교통량 예측 프로젝트에서 LSTM 모델을 선택한 이유와 성능 향상 전략은 무엇이었나요?",
    "Jenkins를 활용한 프로젝트 배포 경험이 있으신데, CI/CD 파이프라인 구축 시 고려한 핵심 요소는 무엇인가요?",
    "OpenCV를 활용한 이미지 전처리에서 가장 중요하게 작용한 필터나 알고리즘은 무엇이었나요?",
    "Tesseract OCR 결과가 불완전할 때, 후처리 모델이 이를 보정하는 흐름은 어떤 방식으로 작동했나요?",
    "알고리즘 테스트에서 B형(Pro) 등급을 취득하셨는데, 시험 대비에서 어떤 전략을 사용하셨나요?",
    "BERT 기반 문장 재구성 모델을 OCR 후처리에 적용할 때 어떤 기준으로 문장 단위를 정의했는지 설명해 주세요.",
    "OCR 전처리 과정에서 적용한 OpenCV 기법들 중 가장 성능 개선에 기여한 기법은 무엇이었고, 그 이유는 무엇인가요?",
    "OCR 결과의 품질 향상을 위해 전처리와 후처리 단계에서 각각 어떤 기술적 전략을 사용하셨나요?",
    "BERT 기반 후처리 모델을 학습할 때 가장 민감하게 반응했던 하이퍼파라미터는 무엇이었나요?",
    "OCR 텍스트가 비정형인 경우에도 BERT가 안정적으로 동작하게 하기 위해 어떤 사전 전처리를 적용하셨나요?",

    # 2. 동기 및 커뮤니케이션
    "큰 규모의 조직을 선호하신다고 했는데, 스타트업과 비교하여 어떤 점이 본인의 성향과 잘 맞는다고 생각하시나요?",
    "문제 해결 중 스트레스를 받는 상황에서 커피나 독서로 기분 전환한다고 하셨는데, 실제 업무 중에도 효과가 있었던 사례가 있나요?",
    "개발자 친화적인 문화에 대해 언급하셨는데, 본인이 생각하는 이상적인 개발문화는 어떤 모습인가요?",
    "코드 리뷰의 중요성을 강조하셨는데, 본인이 리뷰를 주도했던 경험이 있다면 설명해 주세요.",
    "팀 내에서 커뮤니케이션이 잘 된 프로젝트와 그렇지 않았던 프로젝트를 비교해본다면 어떤 차이가 있었나요?",
    "전공 외의 내용을 다루는 업무가 주어졌을 때, 어떻게 접근하시고 어떤 식으로 학습하시나요?",
    "다른 개발자와의 코드 스타일 충돌이 있었던 경험이 있다면, 어떻게 조율하셨나요?",
    "자기주도 학습에 강하다고 하셨는데, 팀 프로젝트 내에서는 어떤 식으로 이를 조화시키셨나요?",
    "협업 시 Jira를 사용해보셨다고 했는데, 업무 분담과 진행 상황 공유를 어떻게 효율화하셨나요?",
    "업무에 몰입하지 못하는 상황에서도 성과를 낸 경험이 있다면 어떻게 극복했는지 이야기해 주세요.",

    # 3. 논리적 사고 및 문제 해결
    "로컬에서는 문제없이 작동하던 서버 코드가 배포 시 에러가 발생한 원인을 어떻게 진단하고 해결하셨나요?",
    "메모리 상의 비트 단위까지 디버깅한 경험이 있으신데, 어떤 판단을 통해 그 수준까지 추적하게 되었나요?",
    "OCR 프로젝트에서 기존 시스템 대비 개선률을 정량적으로 판단한 기준은 무엇이었나요?",
    "LSTM 모델의 MAE를 15% 이하로 낮추는 과정에서 모델 구조 외에 데이터 전처리 측면의 개선이 있었다면요?",
    "Streamlit과 FAISS를 연동하여 이력서 기반 질문을 자동 생성할 때 데이터 검색 정확도를 어떻게 확보했나요?",
    "Flask API 개발 시 논리적 오류나 구조적 비효율을 발견하고 개선한 사례가 있다면 설명해 주세요.",
    "기술부장으로서 팀원 간 기술 수준 차이를 어떻게 인지하고 프로젝트에 반영했는지 설명해 주세요.",
    "정보처리기사 취득 이후 어떤 부분에서 이론이 실무에 도움이 되었는지 구체적으로 말씀해 주세요.",
    "알고리즘 문제 풀이에서 기억나는 가장 어려운 문제는 무엇이었고, 해결 과정에서 얻은 통찰은 무엇인가요?",
    "ORM 학습을 시작하셨다고 했는데, 기존 SQL Mapper(MyBatis)와 비교했을 때 어떤 장단점을 느끼셨나요?"

    # ai_qa.csv
    "개발 중 알 수 없는 버그나 예상치 못한 에러를 해결했던 경험을 구체적으로 말씀해 주세요. 당시 사용했던 디버깅 방법이나 접근 방식을 포함해서 설명해 주세요.",
    "KT AI 연구소 인턴 당시 OCR 문서 처리 시스템을 고도화하셨다고 했는데, 본인이 맡은 역할과 개선 성과를 구체적으로 설명해 주세요.",
    "협업 프로젝트에서 팀원과의 의견 충돌이나 역할 충돌이 있었던 경험이 있나요? 그 상황에서 어떻게 소통하며 조율하셨는지 말씀해 주세요.",
    "본인이 생각하는 이상적인 팀워크란 무엇이고, 그런 팀을 위해 본인은 어떤 노력을 해왔나요?",
    "새로운 기술이나 도구를 배워야 했던 상황에서 어떻게 접근하고 학습하셨는지, 최근에 학습한 기술 중 하나를 예로 들어 말씀해 주세요.",
    "회사를 선택할 때 가장 중요하게 생각하는 기준은 무엇인가요?",
    "팀 내에서 맡은 역할이 본인의 강점과 어떻게 연결되었다고 생각하나요?",
    "OCR 성능을 개선하기 위해 어떤 기술 스택을 선택했으며, 그 이유는 무엇인가요?",
    "프로젝트에서 성능 지표는 어떤 기준으로 설정했고, 목표 달성은 어떻게 평가하셨나요?",
    "기술적 난관에 직면했을 때 어떻게 접근하고 해결하셨나요?",
    "팀 내에서 본인이 기술적으로 가장 많이 기여한 부분은 어떤 부분인가요?",
    "프로젝트 기간 내에 일정과 품질을 동시에 만족시키기 위해 어떤 전략을 사용하셨나요?",
    "새로운 기술을 프로젝트에 도입할 때 어떤 기준으로 기술 스택을 선정하셨나요?",
    "데이터 파이프라인을 설계하면서 고려한 요소와 실제 적용한 방식은 무엇이었나요?",
    "자신이 개발한 기능이 사용자 혹은 내부 프로세스에 어떤 실질적 영향을 주었는지 설명해 주세요.",
    "프로젝트 초기 기획 단계에서 기술적 제안을 한 경험이 있다면 어떤 내용이었나요?",
    "협업 환경에서 Git이나 CI 도구를 사용하며 발생한 이슈를 어떻게 해결하셨나요?",
    "기획자나 디자이너와 협업하면서 생긴 기술적 오해를 어떻게 풀었는지 사례가 있다면 말씀해 주세요.",
    "팀 프로젝트 중 갈등이 생겼을 때 중재하거나 분위기를 전환한 경험이 있다면요?",
    "회의에서 자신의 기술적 의견이 반영되도록 설득한 과정이 있다면 공유해 주세요.",
    "다른 개발자의 코드 스타일을 존중하면서 협업한 구체적인 방법은 무엇이었나요?",
    "동료가 맡은 작업이 지연되었을 때 전체 일정 관점에서 어떻게 대응하셨나요?",
    "코드 리뷰 과정에서 팀원과의 의견 충돌을 조율한 경험이 있다면 공유해 주세요.",
    "기술적 설명을 비전공자나 실무 담당자에게 효과적으로 전달했던 경험이 있나요?",
    "협업 중 예상치 못한 일정 지연이 발생했을 때, 어떻게 대응하고 조율하셨나요?",
    "팀 프로젝트에서 리더와 팔로워 역할 중 어떤 것을 맡았고, 각기 어떤 점이 어려웠나요?",
    "Jira나 GitHub 등 협업 도구를 사용하면서 생긴 커뮤니케이션 노하우가 있다면 설명해주세요.",
    "디버깅 과정에서 일반적인 도구나 방법으로 해결되지 않았던 문제를 어떻게 해결하셨나요?",
    "시스템 병목 현상을 발견하고 해결한 경험이 있다면 어떤 접근을 하셨나요?",
    "성능 튜닝이 필요했던 상황에서 어떤 기준으로 병목을 진단하고 개선하셨나요?",
    "논리적으로 복잡한 문제를 단순하게 분해하여 해결한 경험이 있다면 말씀해 주세요.",
    "이전 버전과 비교하여 개선된 점을 수치나 구조적으로 설명할 수 있나요?",
    "비정형 데이터를 처리하는 과정에서 어떤 문제를 가장 먼저 발견하셨나요?",
    "오류가 반복적으로 발생했을 때, 원인을 어떻게 추적하고 분석하셨나요?",
    "다양한 해결 방법 중 현재 접근 방식을 선택하게 된 판단 근거는 무엇이었나요?",
    "이전 방식 대비 성능 향상을 논리적으로 증명할 수 있는 지표나 실험 결과가 있었나요?",
    "새로운 방법론을 시도했지만 성과가 미흡했던 사례가 있다면, 왜 그런 결과가 나왔다고 생각하시나요?",

]

# Document 리스트로 변환
docs = [Document(page_content=q) for q in question_db]

# 벡터 DB 정의
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_documents(docs, embedding, persist_directory="chroma_db_2")

# Retriever 생성 - 상위 3개 유사질문
retriever = vectorstore.as_retriever(search_kwargs={"k":3})

def generate_question(state: InterviewState) -> InterviewState:
    # 여기에 코드를 완성합니다.
    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

    query_text = f"{state['current_strategy']} {state['resume_keywords']}"
    similar_docs = retriever.invoke(query_text)
    similar_questions = [doc.page_content for doc in similar_docs]

    prompt = PromptTemplate(
            input_variables=["resume_summary",
                             "resume_keywords",
                             "current_strategy",
                             "current_question",
                             "current_answer",
                             "evaluation",
                             "similar_questions"],
            template="""
            질문 전략에 맞는 인터뷰 질문 1개를 생성해야 합니다.
            유사 질문을 꼭 참고하세요.

            [질문전략]
            {current_strategy}

            [유사 질문]
            {similar_questions}

            이력서 요약, 키워드, 이전 질문과 답변, 평가를 기반으로
            지원자의 사고력, 문제 해결 방식,
            혹은 기술적 깊이를 더 확인할 수 있는 심화 인터뷰 질문 1개
            부가적인 말 없이 바로 질문하는 형식으로 생성

            [이력서 요약]
            {resume_summary}

            [키워드]
            {resume_keywords}

            [이전 질문]
            {current_question}

            [이전 답변]
            {current_answer}

            [평가]
            {evaluation}

            """
        )

    # 체인 구성
    chain = prompt | llm

    question = chain.invoke({
        "resume_summary": state['resume_summary'],
        "resume_keywords": state['resume_keywords'],
        "current_strategy": state['current_strategy'],
        "current_question": state['current_question'],
        "current_answer": state['current_answer'],
        "evaluation": state['evaluation'],
        "similar_questions": similar_questions,
    })

    # return 코드는 제공합니다.

    return {
        **state,
        "current_question": question.content,
        "current_answer": ""
    }

# 5) 인터뷰 피드백 보고서 --------------------
def summarize_interview(state: InterviewState) -> InterviewState:
    llm = ChatOpenAI(model_name="gpt-4o-mini")

    resume_summary = state.get("resume_summary", "")
    resume_keywords = ", ".join(state.get("resume_keywords", []))
    conversation = state.get("conversation", [])
    evaluation = state.get("evaluation", [])

    # 인터뷰 로그 문자열 구성
    conversation_block = ""
    for i, (conv, eval_obj) in enumerate(zip(conversation, evaluation)):
        question = conv["question"]
        answer = conv["answer"]
        # Pydantic 모델의 속성은 dot notation으로 접근
        eval_text = f"연관성: {eval_obj.질문과의_관련성}, 구체성: {eval_obj.답변의_구체성}"
        conversation_block += f"[질문 {i+1}] {question}\n[답변 {i+1}] {answer}\n[평가] {eval_text}\n\n"

    # 프롬프트 템플릿
    feedback_prompt = ChatPromptTemplate.from_template("""
        당신은 AI 인터뷰 코치입니다.
        다음은 지원자와의 인터뷰 대화와 각 답변에 대한 평가입니다.

        [이력서 요약]
        {resume_summary}

        [이력서 키워드]
        {resume_keywords}

        [인터뷰 로그 및 평가]
        {conversation_block}

        위 내용을 바탕으로 다음 형식으로 자세한 피드백 보고서를 작성해 주세요:
        1. 전체적인 피드백 요약 (강점, 개선점 요약)
        2. 질문별 상세 피드백 (각 질문에 대한 평가와 개선 조언)
        3. 면접 대비를 위한 구체적인 개선 방향 및 연습 팁

        각 항목은 구분된 제목과 함께 Markdown 형식으로 출력해 주세요.
        """)

    # 프롬프트 인보크 실행
    formatted_prompt = feedback_prompt.invoke({
        "resume_summary": resume_summary,
        "resume_keywords": resume_keywords,
        "conversation_block": conversation_block.strip()
    })

    response = llm.invoke(formatted_prompt)

    ## 1. 수정 부분
    feedback_report = "\n### 인터뷰 피드백 보고서 ###\n" + response.content.strip()
    state['feedback_report'] = feedback_report

    return state

#######################################################
def re_evaluate_answer(state: InterviewState) -> InterviewState:
    print("재평가를 수행합니다.")

    # 무한 루프 방지를 위해 재평가 완료 플래그 설정
    state['re_evaluated'] = True

    # 상태 전이: 재평가 완료 후 다음 단계로 진행
    if state.get('next_step') == 're_evaluate_answer':
        print("재평가 완료. 다음 단계로 진행합니다.")
        state['next_step'] = 'decide_next_step'
    return state

#######################################################

# 6) Agent --------------------
# 분기 판단 함수
def route_next(state: InterviewState) -> Literal["generate", "summarize"]:
    return "summarize" if state["next_step"] == "end" else "generate"
#######################################################
def _eval_branch(state: InterviewState) -> Literal["next", "re"]:
    eval_ok = state.get('evaluation_ok', False)
    re_eval = state.get('re_evaluated', False)
    if not eval_ok and not re_eval:
        return "re"
    return "next"
#######################################################

# 그래프 정의 시작
builder = StateGraph(InterviewState)

# 노드 추가
builder.add_node("evaluate_answer", evaluate_answer)
builder.add_node("decide_next_step", decide_next_step)
builder.add_node("generate_question", generate_question)
builder.add_node("summarize_interview", summarize_interview)
builder.add_node("re_evaluate_answer", re_evaluate_answer) ####

#######################################################
builder.add_conditional_edges("evaluate_answer",_eval_branch,
 {"next":   "decide_next_step",
"re":     "re_evaluate_answer",})

builder.add_edge("re_evaluate_answer", "decide_next_step")

builder.add_conditional_edges("decide_next_step", route_next,
                          {"summarize": "summarize_interview",
                           "generate": "generate_question"}
                          )
#######################################################


# 노드 연결
builder.add_edge(START, "evaluate_answer")
builder.add_edge("summarize_interview", END)
builder.add_edge("generate_question", END)

# 컴파일
graph = builder.compile()


# 그래프 그려보고 싶을때
# print(graph.get_graph().draw_mermaid())
# https://mermaid.live/

#-------------------------------------------------------------------

########### 다음 코드는 제공되는 gradio 코드 입니다.################

import gradio as gr
import tempfile

# 세션 상태 초기화 함수
def initialize_state():
    return {
        "state": None,
        "interview_started": False,
        "interview_ended": False,
        "chat_history": []
    }

# 파일 업로드 후 인터뷰 초기화
def upload_and_initialize(file_obj, session_state):
    if file_obj is None:
        return session_state, "파일을 업로드해주세요."

    # Gradio는 file_obj.name 이 파일 경로야
    file_path = file_obj.name

    # 인터뷰 사전 처리
    state = preProcessing_Interview(file_path)
    session_state["state"] = state
    session_state["interview_started"] = True

    # 첫 질문 저장
    first_question = state["current_question"]
    session_state["chat_history"].append(["🤖 AI 면접관", first_question])

    return session_state, session_state["chat_history"]

# 답변 처리 및 다음 질문 생성
def chat_interview(user_input, session_state):
    if not session_state["interview_started"]:
        return session_state, "먼저 이력서를 업로드하고 인터뷰를 시작하세요."

    # (1) 사용자 답변 저장
    session_state["chat_history"].append(["🙋‍♂️ 지원자", user_input])
    session_state["state"] = update_current_answer(session_state["state"], user_input)

    # (2) Agent 실행 (평가 및 다음 질문 or 종료)
    session_state["state"] = graph.invoke(session_state["state"])

    # (3) 종료 여부 판단
    if session_state["state"]["next_step"] == "end":
        session_state["interview_ended"] = True
        final_summary = "✅ 인터뷰가 종료되었습니다!\n\n"

        ## 1. 수정 부분
        # for i, turn in enumerate(session_state["state"]["conversation"]):
        #     final_summary += f"\n**[질문 {i+1}]** {turn['question']}\n**[답변 {i+1}]** {turn['answer']}\n"
        #     if i < len(session_state["state"]["evaluation"]):
        #         eval_result = session_state["state"]["evaluation"][i]
        #         final_summary += f"_평가 - 질문 연관성: {eval_result.질문과의_관련성}, 답변 구체성: {eval_result.답변의_구체성}_\n"

        session_state["chat_history"].append(["🤖 AI 면접관",
                                              session_state["state"]["feedback_report"]])
        return session_state, session_state["chat_history"]

    else:
        next_question = session_state["state"]["current_question"]
        session_state["chat_history"].append(["🤖 AI 면접관", next_question])
        return session_state, session_state["chat_history"]

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    session_state = gr.State(initialize_state())

    gr.Markdown("# 🤖 AI 면접관 \n이력서를 업로드하고 인터뷰를 시작하세요!")

    with gr.Row():
        file_input = gr.File(label="이력서 업로드 (PDF 또는 DOCX)")
        upload_btn = gr.Button("인터뷰 시작")

    chatbot = gr.Chatbot()
    user_input = gr.Textbox(show_label=False, placeholder="답변을 입력하고 Enter를 누르세요.")

    upload_btn.click(upload_and_initialize, inputs=[file_input, session_state], outputs=[session_state, chatbot])
    user_input.submit(chat_interview, inputs=[user_input, session_state], outputs=[session_state, chatbot])
    user_input.submit(lambda: "", None, user_input)

# 실행
demo.launch(share=True)

Overwriting app.py


# 새 섹션

## **3. 실행**

In [ ]:
!python app.py

/content/app.py:702: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://33f9212cc0fec7a44c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
